<a href="https://colab.research.google.com/github/KevinSikorskiBase2S/machine-learning-intro-workshop/blob/master/TryingSpade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ripped unceremoniously from https://github.com/NVlabs/SPADE

This will take more futzing with to make it work.

In [1]:
!git clone https://github.com/NVlabs/SPADE.git
%cd /content/SPADE/

Cloning into 'SPADE'...
remote: Enumerating objects: 465, done.
remote: Total 465 (delta 0), reused 0 (delta 0), pack-reused 465
Receiving objects: 100% (465/465), 21.93 MiB | 36.33 MiB/s, done.
Resolving deltas: 100% (168/168), done.
/content/SPADE


Sanity check...

In [2]:
!pwd
!ls

/content/SPADE
[Errno 2] No such file or directory: 'SPADE'
/content/SPADE
/content/SPADE
data	  docs	      models   README.md	 test.py   train.py
datasets  LICENSE.md  options  requirements.txt  trainers  util


In [3]:
!pip install -r requirements.txt

In [4]:
%cd models/networks/
!git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
!cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .
%cd ../../

/content/SPADE/models/networks
Cloning into 'Synchronized-BatchNorm-PyTorch'...
remote: Enumerating objects: 158, done.
remote: Total 158 (delta 0), reused 0 (delta 0), pack-reused 158
Receiving objects: 100% (158/158), 34.24 KiB | 1.49 MiB/s, done.
Resolving deltas: 100% (94/94), done.
/content/SPADE


In [0]:
!mkdir checkpoints

Copy checkpoints.tar.gz to /content/SPADE/checkpoints/  (manual step for now)

In [9]:
%cd checkpoints
!tar -xvfz checkpoints.tar.gz
%cd ..
!pwd

/content/SPADE/checkpoints
tar: z: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
/content/SPADE
/content/SPADE


Run test.py

In [0]:
import os
from collections import OrderedDict

import data
from options.test_options import TestOptions
from models.pix2pix_model import Pix2PixModel
from util.visualizer import Visualizer
from util import html

In [0]:
opt = TestOptions().parse()

dataloader = data.create_dataloader(opt)

model = Pix2PixModel(opt)
model.eval()

visualizer = Visualizer(opt)

In [0]:
# create a webpage that summarizes the all results
web_dir = os.path.join(opt.results_dir, opt.name,
                       '%s_%s' % (opt.phase, opt.which_epoch))
webpage = html.HTML(web_dir,
                    'Experiment = %s, Phase = %s, Epoch = %s' %
                    (opt.name, opt.phase, opt.which_epoch))

# test
for i, data_i in enumerate(dataloader):
    if i * opt.batchSize >= opt.how_many:
        break

    generated = model(data_i, mode='inference')

    img_path = data_i['path']
    for b in range(generated.shape[0]):
        print('process image... %s' % img_path[b])
        visuals = OrderedDict([('input_label', data_i['label'][b]),
                               ('synthesized_image', generated[b])])
        visualizer.save_images(webpage, visuals, img_path[b:b + 1])

webpage.save()